# Import Libraries

In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation

# Download the classes of ImageNet dataset

In [ ]:
!wget -O imagnet_classes.txt https://gist.githubusercontent.com/yrevar/942d3a0ac09ec9e5eb3a/raw/238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/imagenet1000_clsidx_to_labels.txt

In [ ]:
classes = open('imagnet_classes.txt').read().strip().split('\n')

In [5]:
# Preprocess image
preprocess = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x[:3, :, :]), # Select only the first 3 channels (RGB)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the image
img_path = 'assets/1.png'  # Replace with your image path
img = Image.open(img_path)
input_img = preprocess(img)   # Add batch dimension

# Grad-CAM

In [23]:
class VGG_19(torch.nn.Module):
    def __init__(self):
        super(VGG_19, self).__init__()

        # get the pretrained VGG19 network
        self.vgg = models.vgg19(pretrained=True)

        # disect the network to access its last convolutional layer(including the ReLU)
        self.features_conv = self.vgg.features[:36]

        # get the max pool of the features stem
        self.max_pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

        # get the average pool
        self.avg_pool = self.vgg.avgpool

        # get the classifier of the vgg19
        self.classifier = self.vgg.classifier

         # placeholder for the gradients
        self.gradients = None


    # hook for the gradients of the activation(feature maps)
    def activations_hook(self, grad):
        self.gradients = grad

    def forward(self, x):
        x = self.features_conv(x)

        # register a backward hook, the hook will be called every
        # time a gradient with respect to the Tensor is computed
        h = x.register_hook(self.activations_hook)

        x = self.max_pool(x)
        x = self.avg_pool(x)
        x = x.view((1, -1)) # one input image a time
        x = self.classifier(x)

        return x

    # method for the gradient extraction
    def get_activations_gradient(self):
        return self.gradients

    # method for the activation exctraction
    def get_activations(self, x):
        return self.features_conv(x)

In [24]:
vgg = VGG_19()
vgg.eval()

/Users/shubbair/miniforge3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/shubbair/miniforge3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /Users/shubbair/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
  0%|          | 1.86M/548M [00:13<1:04:27, 148kB/s] 


KeyboardInterrupt: 